In [1]:
!git clone https://github.com/mateoKutnjak/pose_estimation_pytorch.git

Cloning into 'pose_estimation_pytorch'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 48 (delta 22), reused 43 (delta 17), pack-reused 0
Unpacking objects: 100% (48/48), done.


In [0]:
import sys
sys.path.append('/content/pose_estimation_pytorch/')

In [3]:
import os

import torch
import torch.nn
from torch.utils.data import DataLoader
from torch.optim.rmsprop import RMSprop
from torch.optim.adam import Adam
import argparse

import models
import datasets
import losses
import eval
import util_plot
import loggers

from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
_IMAGES_DIR = '/content/drive/My Drive/_images'
_ANNOTATIONS_JSON_FILE = '/content/drive/My Drive/_annotations/annotations.json'
IMAGES_DIR = '/content/drive/My Drive/images'
ANNOTATIONS_JSON_FILE = '/content/pose_estimation_pytorch/annotations.json'
MEAN_PATH = '/content/drive/My Drive/pose_estimation/mean_total.npy'
STD_PATH = '/content/drive/My Drive/pose_estimation/std_total.npy'
LOGGER_CSV_PATH = '/content/drive/My Drive/pose_estimation/logger.csv'
SAVED_MODEL = '/content/drive/My Drive/pose_estimation/checkpoint.pth'


args_dict = {
    'device': 'cuda', 
    
    'lr': 0.00025, 
    'batch_size': 8, 
    'input_dim': 256, 
    'output_dim': 64, 
    'epochs': 100, 
    'stacks': 2, 
    'channels': 256, 
    'joints': 16, 
    
    'threshold': 0.5, 
    
    'images_dir': IMAGES_DIR, 
    'annots_path': ANNOTATIONS_JSON_FILE, 
    'mean_path': MEAN_PATH, 
    'std_path': STD_PATH, 
    'logger_csv_path': LOGGER_CSV_PATH, 
    'saved_model': SAVED_MODEL
}

In [0]:
if args_dict['saved_model'] is not None and os.path.exists(args_dict['saved_model']):
    device, model, optimizer, saved_args_dict = models.load_model(args_dict['saved_model'])
    args_dict = saved_args_dict
else:
    model = models.HourglassNetwork(
        num_channels=args_dict['channels'],
        num_stacks=args_dict['stacks'],
        num_classes=args_dict['joints'],
        input_shape=(args_dict['input_dim'], args_dict['input_dim'], 3)
    )
    device = torch.device(args_dict['device'])
    model = torch.nn.DataParallel(model).to(device).double()
    optimizer = Adam(model.parameters(), lr=args_dict['lr'])

In [6]:
mpii_train = datasets.MPII_dataset(
        dataset_type='train',
        images_dir=args_dict['images_dir'],
        annots_json_filename=args_dict['annots_path'],
        mean_path=args_dict['mean_path'],
        std_path=args_dict['std_path'],
        input_shape=args_dict['input_dim'],
        output_shape=args_dict['output_dim']
    )

mpii_valid = datasets.MPII_dataset(
    dataset_type='valid',
    images_dir=args_dict['images_dir'],
    annots_json_filename=args_dict['annots_path'],
    mean_path=args_dict['mean_path'],
    std_path=args_dict['std_path'],
    input_shape=args_dict['input_dim'],
    output_shape=args_dict['output_dim']
)

train_dataloader = DataLoader(dataset=mpii_train, batch_size=args_dict['batch_size'], shuffle=True, num_workers=0)
valid_dataloader = DataLoader(dataset=mpii_valid, batch_size=args_dict['batch_size'], shuffle=False, num_workers=0)

criterion = losses.JointsMSELoss().to(device)
logger = loggers.CSVLogger(args_dict['logger_csv_path'])

Train dataset mean and std information found
Mean = [118.84298213 113.28508129 103.7345996 ]
Std = [60.32564024 59.32674718 58.39673706]
Train dataset mean and std information found
Mean = [118.84298213 113.28508129 103.7345996 ]
Std = [60.32564024 59.32674718 58.39673706]


In [0]:
    for epoch in range(args_dict.get('epoch_to_start', 0), args_dict['epochs']):

        model.train()

        for i, (input_batch, output_batch, meta_batch) in enumerate(train_dataloader):

            # TODO adjust learning rate

            x = input_batch.to(device)
            y_kappa = output_batch.to(device, non_blocking=True)
            weights = meta_batch['label_weights'].to(device, non_blocking=True)

            y = model(x)

            loss = 0
            for _y in y:
                loss += criterion(_y, y_kappa, weights)

            joint_distances, accuracy_per_joint, average_accuracy = eval.output_accuracy(
                y=y[-1],
                y_kappa=y_kappa,
                threshold=args_dict['threshold']
            )

            print('TRAIN: Epoch=[{}/{}], Step=[{}/{}], Loss={:.8f}, Avg_Acc: {:.5f}'
                  .format(epoch + 1, args_dict['epochs'], i + 1, len(train_dataloader), loss.item(), average_accuracy))

            logger.log(epoch+1, args_dict['epochs'], i, len(train_dataloader), 'train', loss, accuracy_per_joint, average_accuracy)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()

        with torch.no_grad():

            for i, (input_batch, output_batch, meta_batch) in enumerate(valid_dataloader):
                x = input_batch.to(device)
                y_kappa = output_batch.to(device, non_blocking=True)
                weights = meta_batch['label_weights'].to(device, non_blocking=True)

                y = model(x)

                loss = 0
                for _y in y:
                    loss += criterion(_y, y_kappa, weights)

                joint_distances, accuracy_per_joint, average_accuracy = eval.output_accuracy(
                    y=y[-1],
                    y_kappa=y_kappa,
                    threshold=args_dict['threshold']
                )

                print('VALID: Epoch=[{}/{}], Step=[{}/{}], Loss={:.8f}, Avg_Acc: {:.5f}'
                      .format(epoch + 1, args_dict['epochs'], i + 1, len(valid_dataloader), loss.item(),
                              average_accuracy))

                logger.log(epoch+1, args_dict['epochs'], i, len(train_dataloader), 'valid', loss, accuracy_per_joint, average_accuracy)

        args_dict['epoch_to_start'] = epoch+1

        models.save_model(
            model=model,
            optimizer=optimizer,
            args_dict=args_dict
        )

TRAIN: Epoch=[1/100], Step=[1/2781], Loss=0.04099230, Avg_Acc: 0.01562
TRAIN: Epoch=[1/100], Step=[2/2781], Loss=0.03181084, Avg_Acc: 0.03125
TRAIN: Epoch=[1/100], Step=[3/2781], Loss=0.02106292, Avg_Acc: 0.00000
TRAIN: Epoch=[1/100], Step=[4/2781], Loss=0.01978947, Avg_Acc: 0.00000
TRAIN: Epoch=[1/100], Step=[5/2781], Loss=0.01603642, Avg_Acc: 0.03125
TRAIN: Epoch=[1/100], Step=[6/2781], Loss=0.01461723, Avg_Acc: 0.03125
TRAIN: Epoch=[1/100], Step=[7/2781], Loss=0.01305991, Avg_Acc: 0.02344
TRAIN: Epoch=[1/100], Step=[8/2781], Loss=0.01039232, Avg_Acc: 0.01562
TRAIN: Epoch=[1/100], Step=[9/2781], Loss=0.00959568, Avg_Acc: 0.00781
TRAIN: Epoch=[1/100], Step=[10/2781], Loss=0.00948768, Avg_Acc: 0.01562
TRAIN: Epoch=[1/100], Step=[11/2781], Loss=0.00679757, Avg_Acc: 0.01562
TRAIN: Epoch=[1/100], Step=[12/2781], Loss=0.00623928, Avg_Acc: 0.02344
TRAIN: Epoch=[1/100], Step=[13/2781], Loss=0.00583253, Avg_Acc: 0.04688
TRAIN: Epoch=[1/100], Step=[14/2781], Loss=0.00434230, Avg_Acc: 0.05469
T